# Task 5: Image Classification using Convolutional Neural Networks

In this project our aim was to classify images using Convolutional Neural Networks, to do so we used the Inception-V3 Network, which is trained for the ImageNet Large Visual Recognition Challenge using the data from 2012. Furthermore this model is able to achieve 76.88% Top-1 Accuracy and 93.344% Top-5 Accuracy on ILSVRC2012-Validation Set.

To do so, we used R and MXNet, generating an interactive user interface with Shiny where the user can upload images in oder to classify them. Hence, first of all we had to load all the required packages (libraries):


In [1]:
required_packages <- c("shiny", "mxnet", "imager", "scales", "jpeg", "ggplot2", "readr", "png", "ggthemes")
sapply(required_packages, require, character.only = TRUE)

Loading required package: shiny
Loading required package: mxnet
Loading required package: imager
Loading required package: plyr
Loading required package: magrittr

Attaching package: ‘imager’

The following object is masked from ‘package:magrittr’:

    add

The following object is masked from ‘package:plyr’:

    liply

The following objects are masked from ‘package:stats’:

    convolve, spectrum

The following object is masked from ‘package:graphics’:

    frame

The following object is masked from ‘package:base’:

    save.image

Loading required package: scales
Loading required package: jpeg
Loading required package: ggplot2
Loading required package: readr

Attaching package: ‘readr’

The following object is masked from ‘package:scales’:

    col_factor

Loading required package: png
Loading required package: ggthemes


shiny    mxnet   imager   scales     jpeg  ggplot2    readr      png 
    TRUE     TRUE     TRUE     TRUE     TRUE     TRUE     TRUE     TRUE 
ggthemes 
    TRUE

The Inception-V3 Network has some constraints regarding the image input dimensions which need to be single crop on 299 x 299 image, to do so we defined a preprocessing function, which preprocesses the images before inputting them in the Network model, below we can find the code for the preprocessing (an adaptation of the code provided in the inception v3 model but translated to R):

In [2]:
preproc.image <- function(image) {
  

  shape <- dim(image)
  short.edge <- min(shape[1:2])
  xx <- floor((shape[1] - short.edge) / 2)
  yy <- floor((shape[2] - short.edge) / 2) 
  croped <- crop.borders(image, xx, yy)
  resized <- resize(croped, 299, 299)
  array <- as.array(resized) * 256
  dim(array) <- c(299, 299, 3)
  preprocessedImage <- array - 128
  preprocessedImage <- array/128
  
  dim(preprocessedImage) <- c(299, 299, 3, 1)
  return(preprocessedImage)
}


Once we defined the preprocessing function, we can proceed to download the model and define the model and the classes used in this model. To do so, we will check if the file "model", which is the name of the folder where Inception V3 model is contained is found in the computer. If it is not found, we will proceed to download and untar the model, and then we will load the model to our system and also the classes (which are in txt format).

In [3]:
if (!file.exists("model")) {
    download.file("http://data.dmlc.ml/mxnet/models/imagenet/inception-v3.tar.gz", destfile = "inception-v3.tar.gz")
    untar("inception-v3.tar.gz")
  }
  
  ImageNet_Model <- mx.model.load("model/Inception-7", iteration = 1)
  
  synsets <- read_lines("model/synset.txt")

After defining the preprocessing function we also defined a function to pick an image from some location (in the web or in the local computer):

In [4]:
picksource <- function(input){
  
  ImageFromUrl <- eventReactive(input$enter, {
    print(input$urlImage)
   
      image <- tempfile()
      download.file(input$urlImage, destfile = image, method = "auto")
      return(image)
    
  })
  
  
  src = if (input$tabs == "Upload Image") {
    if (is.null(input$Imagefile)) {
      if (input$goButton == 0 || is.null(ImageFromUrl())) {
        'image.jpg'
      } else {
        ImageFromUrl()
      }
    } else {
      input$Imagefile$datapath
    }
  } else {
    if (input$goButton == 0 || is.null(ImageFromUrl())) {
      if (is.null(input$Imagefile)) {
        'image.jpg'
      } else {
        input$Imagefile$datapath
      }
    } else {
      ImageFromUrl()
    }
  }
}

Furthermore, we will define a function to appropiately format the results, formatting the final results in a nice and visual way.

In [5]:
getCommonResults <- function(result){
  StringResult <- ""
  for (i in 1:5) {
    ResultInJ <- strsplit(result[i], " ")[[1]]
    for (j in 2:length(ResultInJ)) {
      StringResult <- paste(StringResult, ResultInJ[j])
    }
    StringResult <- paste(StringResult, "\n")
  }
  StringResult

}

Now that we have defined all the previous functions, we will define our shiny user interface, in this user inferface we defined a side by side layout, where we defined in one side a panel to upload an image from the computer or from an url. And in the other side we defined a panel where the results will be displayed, displaying the inputted image, the predicted classes and its probabilities.

In [6]:
# Define UI for application that draws a histogram
ui <- fluidPage(
  
  includeCSS('alike.css'),
  # Application title
  titlePanel("Image Classification using Convolutional Neural Networks"),
  
  # Sidebar with a slider input for number of bins 
  sidebarLayout(
    sidebarPanel(
      tabsetPanel(
        id = "tabs",
        tabPanel("Upload Image",
                 fileInput('Imagefile', '\n Upload an Image:')),
        tabPanel(
          "Use the URL",
          textInput("urlImage", "Enter an Image URL:", ""),
          actionButton("enter", "OK")
        )
      )
    ),
    
    # Show the image, the results and the probabilities
    mainPanel(
      h3("Image to classify"),
      tags$hr(),
      imageOutput("imageloaded", height = "auto"),
      tags$hr(),
      h3("What does the image contains?"),
      tags$hr(),
      verbatimTextOutput("res"),
      plotOutput("ProbPlot")
    )
  )
)

Furthermore, we had to define the actions behind the user interface, i.e. define what each interface does. To do so we generated a server, where we defined the logic required to pick an image submitted by the user and predict its class.

In [9]:
server <- function(input, output) {
  
  image <- NULL
  
  
  
  
  output$imageloaded = renderImage(list(src = picksource(input)), deleteFile = FALSE)
  
  
  
  output$res <- renderText({
    src <- picksource(input)
    
    im <- load.image(src)
    preprocessedImage <- preproc.image(im)
    prob <- predict(ImageNet_Model, X = preprocessedImage)
    max.idx <- order(prob[,1], decreasing = TRUE)[1:5]
    result <- synsets[max.idx]
    StringResult <- ""
    for (i in 1:5) {
      ResultInJ <- strsplit(result[i], " ")[[1]]
      for (j in 2:length(ResultInJ)) {
        StringResult <- paste(StringResult, ResultInJ[j])
      }
      StringResult <- paste(StringResult, "\n")
    }
    StringResult
  })
  
  
  
  output$res <- renderText({
    src <- picksource(input)
    
    im <- load.image(src)
    preprocessedImage <- preproc.image(im)
    prob <- predict(ImageNet_Model, X = preprocessedImage)
    max.idx <- order(prob[,1], decreasing = TRUE)[1:5]
    result <- synsets[max.idx]
    StringResult <- StringResult <- getCommonResults(result)
  })
  
  output$ProbPlot <- renderPlot({
   src <- picksource(input)
    
    im <- load.image(src)
    preprocessedImage <- preproc.image(im)
    prob <- predict(ImageNet_Model, X = preprocessedImage)
    max.idx <- order(prob[,1], decreasing = TRUE)[1:5]
    result <- synsets[max.idx]
    StringResult <- getCommonResults(result)
    
    
    StringResult1 <- unlist(strsplit(StringResult, split="\n"))
    StringResult1 <- data.frame(names = StringResult1, probability = sort(prob[,1], decreasing = TRUE)[1:5])
    ggplot(StringResult1) + geom_bar(aes(reorder(names, probability ), probability * 100), stat = "identity", fill = "cyan", alpha = 1/3) + coord_flip() +
      labs(x = "Names", y = "Probability (%)") + theme_solarized_2(light = FALSE) + theme_hc(bgcolor = "darkunica") +
      scale_colour_hc("darkunica")
  })
  
  
}

After defining the user and server, we can run the application by running the following chunck:

In [ ]:
shinyApp(ui = ui, server = server)


Listening on http://127.0.0.1:5759


In [ ]:
Note: In order to access to the application click on the url generated after running the previous chunk.

## References
[1] Szegedy, Christian, et al. "Rethinking the Inception Architecture for Computer Vision." arXiv preprint arXiv:1512.00567 (2015).